In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
# Crie uma sessão Spark
spark = SparkSession.builder \
    .appName("tech-challenge-fase3") \
    .getOrCreate()

24/08/18 17:36:34 WARN Utils: Your hostname, spark-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/08/18 17:36:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/08/18 17:36:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/18 17:36:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [34]:
# Ler os arquivos Parquet
df_data = spark.read.parquet("./data/parquet/tbx001_data.parquet")
df_states = spark.read.parquet("./data/parquet/tbx002_states.parquet")
df_gender = spark.read.parquet("./data/parquet/tbx003_gender.parquet")
df_residential_area = spark.read.parquet("./data/parquet/tbx004_residential_area.parquet")

In [36]:
# Registrar os DataFrames como tabelas temporárias
df_data.createOrReplaceTempView("tbx001_data")
df_states.createOrReplaceTempView("tbx002_states")
df_gender.createOrReplaceTempView("tbx003_gender")
df_residential_area.createOrReplaceTempView("tbx004_residential_area")

In [37]:
# Realizar o join entre as tabelas usando SQL
query = """
SELECT 
    t1.uf,
    t1.semana_mes,
    t1.mes,
    t1.area_domicilio,
    t1.idade,
    t2.name as sexo,
    t1.cor_raca,
    t1.escolaridade,
    t1.teve_febre,
    t1.teve_dificuldade_respirar,
    t1.teve_dor_cabeca,
    t1.teve_fadiga,
    t1.teve_perda_cheiro,
    t1.foi_posto_saude,
    t1.ficou_em_casa,
    t1.ficou_internado,
    t1.tem_plano_saude,
    t1.assalariado,
    t1.faixa_rendimento,
    t1.situacao_domicilio,
    t1.resultado_covid
FROM tbx001_data t1
JOIN tbx003_gender t2
ON t1.sexo = t2.id
"""
# Executar a consulta SQL
df_joined = spark.sql(query)

In [38]:
# Exibir o resultado do join
df_joined.show()

+---+----------+---+--------------+-----+---------+--------+------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+-----------+----------------+------------------+---------------+
| uf|semana_mes|mes|area_domicilio|idade|     sexo|cor_raca|escolaridade|teve_febre|teve_dificuldade_respirar|teve_dor_cabeca|teve_fadiga|teve_perda_cheiro|foi_posto_saude|ficou_em_casa|ficou_internado|tem_plano_saude|assalariado|faixa_rendimento|situacao_domicilio|resultado_covid|
+---+----------+---+--------------+-----+---------+--------+------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+-----------+----------------+------------------+---------------+
| 11|         4| 06|             1|  035|Masculino|       4|           5|         2|                        2|              2|       NULL|             